In [3]:
import numpy as np
import random
import os, errno
import sys
from tqdm import trange


#torch.manual_seed(101)
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
torch.manual_seed(101)
class RMSLELoss(nn.Module):
    def __init__(self):
        super().__init__()
        self.mse = nn.MSELoss()
        
    def forward(self, pred, actual):
        return torch.sqrt(self.mse(torch.log(pred + 1), torch.log(actual + 1)))
    
    
class lstm_encoder(nn.Module):
    ''' Encodes time-series sequence '''

    def __init__(self, input_size, hidden_size, num_layers = 3):
        
        '''
        : param input_size:     the number of features in the input X
        : param hidden_size:    the number of features in the hidden state h
        : param num_layers:     number of recurrent layers (i.e., 2 means there are
        :                       2 stacked LSTMs)
        '''
        
        super(lstm_encoder, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        # define LSTM layer
        self.lstm = nn.LSTM(input_size = input_size, hidden_size = hidden_size,
                            num_layers = num_layers)

    def forward(self, x_input):
        
        '''
        : param x_input:               input of shape (seq_len, # in batch, input_size)
        : return lstm_out, hidden:     lstm_out gives all the hidden states in the sequence;
        :                              hidden gives the hidden state and cell state for the last
        :                              element in the sequence 
        '''
        
        lstm_out, self.hidden = self.lstm(x_input.view(x_input.shape[0], x_input.shape[1], self.input_size))
        
        return lstm_out, self.hidden     
    
    def init_hidden(self, batch_size):
        
        '''
        initialize hidden state
        : param batch_size:    x_input.shape[1]
        : return:              zeroed hidden state and cell state 
        '''
        
        return (torch.zeros(self.num_layers, batch_size, self.hidden_size, device='cuda'),
                torch.zeros(self.num_layers, batch_size, self.hidden_size, device='cuda'))


class lstm_decoder(nn.Module):
    ''' Decodes hidden state output by encoder '''
    
    def __init__(self, input_size, hidden_size, num_layers = 3):

        '''
        : param input_size:     the number of features in the input X
        : param hidden_size:    the number of features in the hidden state h
        : param num_layers:     number of recurrent layers (i.e., 2 means there are
        :                       2 stacked LSTMs)
        '''
        
        super(lstm_decoder, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        self.lstm = nn.LSTM(input_size = input_size, hidden_size = hidden_size,
                            num_layers = num_layers)
        self.linear = nn.Linear(hidden_size, input_size)           

    def forward(self, x_input, encoder_hidden_states):
        
        '''        
        : param x_input:                    should be 2D (batch_size, input_size)
        : param encoder_hidden_states:      hidden states
        : return output, hidden:            output gives all the hidden states in the sequence;
        :                                   hidden gives the hidden state and cell state for the last
        :                                   element in the sequence 
 
        '''
        
        lstm_out, self.hidden = self.lstm(x_input.unsqueeze(0), encoder_hidden_states)
        output = self.linear(lstm_out.squeeze(0))     
        
        return output, self.hidden

class lstm_seq2seq(nn.Module):
    ''' train LSTM encoder-decoder and make predictions '''
    
    def __init__(self, input_size, hidden_size):

        '''
        : param input_size:     the number of expected features in the input X
        : param hidden_size:    the number of features in the hidden state h
        '''

        super(lstm_seq2seq, self).__init__()

        self.input_size = input_size
        self.hidden_size = hidden_size

        self.encoder = lstm_encoder(input_size = input_size, hidden_size = hidden_size).cuda()
        self.decoder = lstm_decoder(input_size = input_size, hidden_size = hidden_size).cuda()
        

    def train_model(self, input_tensor, target_tensor, n_epochs, target_len, batch_size, training_prediction = 'recursive', teacher_forcing_ratio = 0.5, learning_rate = 0.01, dynamic_tf = False):
        
        '''
        train lstm encoder-decoder
        
        : param input_tensor:              input data with shape (seq_len, # in batch, number features); PyTorch tensor    
        : param target_tensor:             target data with shape (seq_len, # in batch, number features); PyTorch tensor
        : param n_epochs:                  number of epochs 
        : param target_len:                number of values to predict 
        : param batch_size:                number of samples per gradient update
        : param training_prediction:       type of prediction to make during training ('recursive', 'teacher_forcing', or
        :                                  'mixed_teacher_forcing'); default is 'recursive'
        : param teacher_forcing_ratio:     float [0, 1) indicating how much teacher forcing to use when
        :                                  training_prediction = 'teacher_forcing.' For each batch in training, we generate a random
        :                                  number. If the random number is less than teacher_forcing_ratio, we use teacher forcing.
        :                                  Otherwise, we predict recursively. If teacher_forcing_ratio = 1, we train only using
        :                                  teacher forcing.
        : param learning_rate:             float >= 0; learning rate
        : param dynamic_tf:                use dynamic teacher forcing (True/False); dynamic teacher forcing
        :                                  reduces the amount of teacher forcing for each epoch
        : return losses:                   array of loss function for each epoch
        '''
        
        #
        
        # initialize array of losses 
        losses = np.full(n_epochs, np.nan)

        optimizer = optim.Adam(self.parameters(), lr = learning_rate)
        criterion = nn.MSELoss().cuda()
        
        # calculate number of batch iterations
        n_batches = int(input_tensor.shape[1] / batch_size)

        with trange(n_epochs) as tr:
            for it in tr:
                
                batch_loss = 0.
                batch_loss_tf = 0.
                batch_loss_no_tf = 0.
                num_tf = 0
                num_no_tf = 0

                for b in range(n_batches):
                    # select data 
                    input_batch = input_tensor[:, b: b + batch_size, :].cuda()
                    target_batch = target_tensor[:, b: b + batch_size, :].cuda()
                    # outputs tensor
                    outputs = torch.zeros(target_len, batch_size, input_batch.shape[2])

                    # initialize hidden state
                    encoder_hidden = self.encoder.init_hidden(batch_size)

                    # zero the gradient
                    optimizer.zero_grad()

                    # encoder outputs
                    encoder_output, encoder_hidden = self.encoder(input_batch)

                    # decoder with teacher forcing
                    decoder_input = input_batch[-1, :, :]   # shape: (batch_size, input_size)
                    decoder_hidden = encoder_hidden

                    if training_prediction == 'recursive':
                        # predict recursively
                        for t in range(target_len): 
                            decoder_output, decoder_hidden = self.decoder(decoder_input, decoder_hidden)
                            outputs[t] = decoder_output
                            decoder_input = decoder_output

                    if training_prediction == 'teacher_forcing':
                        # use teacher forcing
                        if random.random() < teacher_forcing_ratio:
                            for t in range(target_len): 
                                decoder_output, decoder_hidden = self.decoder(decoder_input, decoder_hidden)
                                outputs[t] = decoder_output
                                decoder_input = target_batch[t, :, :]

                        # predict recursively 
                        else:
                            for t in range(target_len): 
                                decoder_output, decoder_hidden = self.decoder(decoder_input, decoder_hidden)
                                outputs[t] = decoder_output
                                decoder_input = decoder_output

                    if training_prediction == 'mixed_teacher_forcing':
                        # predict using mixed teacher forcing
                        for t in range(target_len):
                            decoder_output, decoder_hidden = self.decoder(decoder_input, decoder_hidden)
                            outputs[t] = decoder_output
                            
                            # predict with teacher forcing
                            if random.random() < teacher_forcing_ratio:
                                decoder_input = target_batch[t, :, :]
                            
                            # predict recursively 
                            else:
                                decoder_input = decoder_output

                    # compute the loss 
                    loss = criterion(outputs.cuda(), target_batch.cuda())
                    batch_loss += loss.item()
                    
                    # backpropagation
                    loss.backward()
                    optimizer.step()

                # loss for epoch 
                batch_loss /= n_batches 
                losses[it] = batch_loss

                # dynamic teacher forcing
                if dynamic_tf and teacher_forcing_ratio > 0:
                    teacher_forcing_ratio = teacher_forcing_ratio - 0.02 

                # progress bar 
                tr.set_postfix(loss="{0:.3f}".format(batch_loss))
                    
        return losses

    def predict(self, input_tensor, target_len):
        
        '''
        : param input_tensor:      input data (seq_len, input_size); PyTorch tensor 
        : param target_len:        number of target values to predict 
        : return np_outputs:       np.array containing predicted values; prediction done recursively 
        '''

        # encode input_tensor
        input_tensor = input_tensor.unsqueeze(1)     # add in batch size of 1
        encoder_output, encoder_hidden = self.encoder(input_tensor)

        # initialize tensor for predictions
        outputs = torch.zeros(target_len, input_tensor.shape[2])

        # decode input_tensor
        decoder_input = input_tensor[-1, :, :]
        decoder_hidden = encoder_hidden
        
        for t in range(target_len):
            decoder_output, decoder_hidden = self.decoder(decoder_input, decoder_hidden)
            outputs[t] = decoder_output.squeeze(0)
            decoder_input = decoder_output
            
        np_outputs = outputs.detach().numpy()
        
        return np_outputs


In [4]:
import pandas as pd

def load_feather(path):
    return pd.read_feather(path)

def load_csv(path):
    return pd.read_csv(path)

In [6]:
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import numpy as np


def remove_max(x):
    x[x.argmax()] = np.median(x)
    print(x.argmax(), ":", x[x.argmax()], "=>", np.median(x))
    return x

def groupby_datapoint(df,
                      gb='YYYYMMDD',
                      target='Qty'):
    
    df_ts = df.groupby([gb])[target].sum().sort_index()
    df_ts = df_ts.reset_index()
    #df_ts[gb] = pd.to_datetime(df_ts[gb], format='%Y%m%d')
    df_ts[gb] = df_ts[gb].astype(int)

    df_ts.set_index([gb], inplace=True)
    df_ts.sort_index(inplace=True)
    return df_ts 


def windowed_dataset(y, input_window = 5, output_window = 1, stride = 1, num_features = 1):
  
    '''
    create a windowed dataset
    
    : param y:                time series feature (array)
    : param input_window:     number of y samples to give model 
    : param output_window:    number of future y samples to predict  
    : param stide:            spacing between windows   
    : param num_features:     number of features (i.e., 1 for us, but we could have multiple features)
    : return X, Y:            arrays with correct dimensions for LSTM
    :                         (i.e., [input/output window size # examples, # features])
    '''
  
    L = y.shape[0]
    num_samples = (L - input_window - output_window) // stride + 1

    X = np.zeros([input_window, num_samples, num_features])
    Y = np.zeros([output_window, num_samples, num_features])    
    
    for ff in np.arange(num_features):
        for ii in np.arange(num_samples):
            start_x = stride * ii
            end_x = start_x + input_window
            X[:, ii, ff] = y[start_x:end_x, ff]

            start_y = stride * ii + input_window
            end_y = start_y + output_window 
            Y[:, ii, ff] = y[start_y:end_y, ff]

    return X, Y


def numpy_to_torch(x):
    return torch.from_numpy(x).type(torch.Tensor)



In [7]:
    DATA_PATH = 'Tire_edit.csv'
    MODEL_PATH = 'tire.pth'
    RESULT_PATH = 'sample_submission.csv'

    EPOCHS = 85
    BATCH_SIZE = 512
    LR = 0.005
   
    IN_WINDOW_SIZE = 7*4
    OUT_WINDOW_SIZE = 31
    STRIDE = 1
    NUM_FEATURE = 1
    TARGET_COL = 'Qty'    
    
    HIDDEN_SIZE = 128
    
    # Load data
    train_df = load_csv(DATA_PATH)
    train_df = train_df.drop([train_df.columns[0]],axis=1)
    #print(train_df.shape)
    #train_df = train_df[:1216]

    #print(train_df.iloc[1215])
    # Preprocessing
    train_amt = train_df[TARGET_COL].values.reshape(-1, 1)
    #print(len(train_amt))
    # scaling
#     train_amt = remove_max(train_amt) # convert max value to median
#     train_amt = remove_max(train_amt) # convert max value to median
#     train_amt = remove_max(train_amt) # convert max value to median
#     train_amt = remove_max(train_amt) # convert max value to median
#     train_amt = remove_max(train_amt) # convert max value to median
#     train_amt = remove_max(train_amt) # convert max value to median
    scaler = MinMaxScaler().fit(train_amt)
    train_amt = scaler.transform(train_amt)
    #print(train_amt.shape)
    #print(train_amt.shape)
    # split x_train, x_test
    x_train = train_amt[:-IN_WINDOW_SIZE] # Input for train
    x_test = train_amt[-IN_WINDOW_SIZE:]  # Input for test prediction
    
    # Make sequence
    x_train, y_train = windowed_dataset(x_train,
                                        IN_WINDOW_SIZE,
                                        OUT_WINDOW_SIZE,
                                        STRIDE,
                                        NUM_FEATURE)

    # to torch
    x_train = numpy_to_torch(x_train)
    y_train = numpy_to_torch(y_train)
    x_test = numpy_to_torch(x_test)

In [8]:
    model = lstm_seq2seq(input_size = 1, hidden_size = HIDDEN_SIZE).cuda()
    model.load_state_dict(torch.load(MODEL_PATH))
    model.cuda()
    
    y_test_pred = model.predict(x_test.cuda(), target_len = OUT_WINDOW_SIZE)
    #print(y_test_pred)

In [9]:
y_test_pred1 = y_test_pred[-28:]
y_test_pred1 = numpy_to_torch(y_test_pred1)
y_test_pred2 = model.predict(y_test_pred1.cuda(), target_len = OUT_WINDOW_SIZE)

In [10]:
y_test_pred3 = y_test_pred2[-29:-1]
print(len(y_test_pred3))
y_test_pred3 = numpy_to_torch(y_test_pred3)
y_test_pred4 = model.predict(y_test_pred3.cuda(), target_len = OUT_WINDOW_SIZE)

28


In [11]:
y_test_pred = scaler.inverse_transform(y_test_pred)
y_test_pred2 = scaler.inverse_transform(y_test_pred2)
y_test_pred4 = scaler.inverse_transform(y_test_pred4)

In [12]:
y_test_pred = y_test_pred.tolist()
y_test_pred2 = y_test_pred2.tolist()
y_test_pred4 = y_test_pred4.tolist()

In [13]:
del y_test_pred[26]
del y_test_pred[25]
del y_test_pred[19]
del y_test_pred[18]
del y_test_pred[12]
del y_test_pred[11]
del y_test_pred[5]


del y_test_pred2[30]
del y_test_pred2[23]
del y_test_pred2[16]
del y_test_pred2[9]
del y_test_pred2[8]
del y_test_pred2[2]
del y_test_pred2[1]

del y_test_pred4[28]
del y_test_pred4[27]
del y_test_pred4[25]
del y_test_pred4[24]
del y_test_pred4[23]
del y_test_pred4[21]
del y_test_pred4[20]
del y_test_pred4[14]
del y_test_pred4[13]
del y_test_pred4[7]
del y_test_pred4[6]
del y_test_pred4[0]



In [14]:
finalout =[]

finalout.extend(y_test_pred)
finalout.extend(y_test_pred2)
finalout.extend(y_test_pred4)

finalout = sum(finalout, [])


print(len(finalout))

67


In [15]:
    sub_sample = pd.read_csv("/home/workspace/data/.train/.task152/submission_sample.csv")
    len_ans = sub_sample.shape[0]
    sub_sample['Qty'] = finalout[:len_ans]
#     for qwe in range(0,23):
#         sub_sample.iloc[qwe,1] = sub_sample.iloc[qwe,1]
        
    for qwe in range(52,67):
        sub_sample.iloc[qwe,1] = sub_sample.iloc[qwe,1]*0.6
        
    for qwe in range(58,67):
        sub_sample.iloc[qwe,1] = sub_sample.iloc[qwe,1]*0.5
    
    for qwe in range(62,67):
        sub_sample.iloc[qwe,1] = sub_sample.iloc[qwe,1]*0.5
        
    sub_sample.iloc[2,1] = 50
    sub_sample.iloc[4,1] = 50
    sub_sample.iloc[35,1] = 50
    sub_sample.iloc[41,1] = 50
    sub_sample.iloc[47,1] = 50   
    #sub_sample.iloc[23,1] = sub_sample.iloc[23,1]*1.2
    
    sub_sample.to_csv(RESULT_PATH, index=False)
    
    print(sub_sample)
        

    YYYYMMDD           Qty
0   20191001  27969.736328
1   20191002  22318.718750
2   20191003     50.000000
3   20191004  23526.933594
4   20191005     50.000000
..       ...           ...
62  20191220   4195.848926
63  20191223   4087.154297
64  20191227   4008.524414
65  20191230   4062.691406
66  20191231   3513.066797

[67 rows x 2 columns]


In [16]:
from nipa.taskSubmit import nipa_submit
import os

team_id = "1327"
task_no= "152"
prediction_path = RESULT_PATH
# 파일 존재 여부 확인
print("is file: ", os.path.isfile(prediction_path))

is file:  True


In [17]:
# 제출 성공
nipa_submit(team_id=team_id,
task_no=task_no,
result=prediction_path
model = 
)

20201126221555856783_mdUG.csv: 200
